# Query book

In [ ]:
import lamindb as ln

ln.track()

## Basic select operations

### Output formats

In [ ]:
ln.select(ln.File).df().head()

In [ ]:
ln.select(ln.File).all()[:3]

In [ ]:
ln.select(ln.File).first()

### and

In [ ]:
ln.select(ln.File, name="Mouse Lymph Node scRNA-seq", suffix=".h5ad").one()

In [ ]:
ln.select(ln.File).where(
    ln.File.name == "Mouse Lymph Node scRNA-seq", ln.File.suffix == ".h5ad"
).one()

### or

In [ ]:
ln.select(ln.File).where((ln.File.suffix == ".zarr") | (ln.File.suffix == ".h5ad")).df()

### in

In [ ]:
ln.select(ln.File).where(ln.File.suffix.in_([".zarr", ".h5ad"])).df()

### order by

In [ ]:
ln.select(ln.Run).order_by(ln.Run.created_at.desc()).df()

### contains

In [ ]:
ln.select(ln.Transform).where(ln.Transform.title.contains("lineage")).df()

### startswith

In [ ]:
ln.select(ln.Transform).where(ln.Transform.title.contains("Track")).df()

## Data lineage

### Track run, inputs & outputs

From which run does file `output.bam` come from?

In [ ]:
with ln.Session() as ss:
    file = ss.select(ln.File, name="output.bam").one()
    print(file.run)

Which other files did this run have as input and outputs?

In [ ]:
with ln.Session() as ss:
    file = ss.select(ln.File, name="output.bam").one()
    print(file.run.inputs)
    print(file.run.outputs)

### Track notebook

Which notebook ingested the dataset `Mouse Lymph Node scRNA-seq`?

In [ ]:
with ln.Session() as ss:
    file = ss.select(ln.File, name="Mouse Lymph Node scRNA-seq").one()
    print(file.transform)

Which notebooks were created by testuser2?

In [ ]:
ln.select(ln.Transform, type="notebook").join(ln.Transform.created_by).where(
    ln.User.handle == "testuser2"
).df()

### Track pipeline

Which pipeline produced this file `input.fastq.gz`?

In [ ]:
with ln.Session() as ss:
    file = ss.select(ln.File, name="input.fastq.gz").one()
    print(file.transform)

Which pipelines were created by testuser1?

In [ ]:
ln.select(ln.Transform, type="pipeline").join(ln.Transform.created_by).where(
    ln.User.handle == "testuser1"
).df()

### Track user

Which users have interacted with the database?

In [ ]:
ln.select(ln.User).df()

Which user ingested this file `input.fastq.gz`?

In [ ]:
with ln.Session() as ss:
    file = ss.select(ln.File, name="input.fastq.gz").one()
    print(file.created_by)

Which users created notebooks with "lineage" in the title?

In [ ]:
ln.select(ln.User.handle, ln.Transform.title).join(ln.Transform.created_by).where(
    ln.Transform.title.contains("lineage")
).df()

Which user created this pipeline?

In [ ]:
ln.select(ln.Transform).df()

In [ ]:
ln.select(ln.User).join(ln.Transform, name="Cell Ranger", version="7").one()